# This notebook are for testing some Linear algebra 


In [1]:
import numpy as np
import scipy.linalg as sl
from scipy.linalg import eigh, qr, svd

### Build some matrices for testing

In [2]:
H=np.array([[1.68366952, 1.59227351, 1.06408247],
       [1.59227351, 1.60517898, 1.1263274 ],
       [1.06408247, 1.1263274 , 0.15138217]])

S=np.array([[1, 0.1, 0.3],[0.1, 1, 0.05],[0.3, 0.05, 1]])
nb=6
H=np.random.random((nb,nb))
H=H+H.T.conj()



#### Test orthogonal eigen problem
##### Testing :
$H\Phi = \Phi E$ 

$H\Phi \neq E\Phi$

$\Phi E \Phi^\dagger = H$

$\Phi \Phi^\dagger = I$

In [3]:
E, Phi =sl.eigh(H)
print(np.isclose(H@Phi, np.diag(E)@Phi).all())
print(np.isclose(H@Phi, Phi@np.diag(E)).all())
print(np.isclose(Phi @ np.diag(E) @Phi.T.conj(), H).all())
print(np.isclose( Phi.T.conj()@Phi,np.eye(nb)).all())

False
True
True
True


### Test Scdmk

In [119]:
def scdm(psiT, ncol):
    """
    select columns for a psiT.
    """
    _Q, _R, piv = qr(psiT, mode='full', pivoting=True)
    cols = piv[:ncol]
    return cols

def get_Amn(psiT):
    U, S, VT= svd(psiT, full_matrices=False)
    Amn=U@VT
    return Amn

def get_wann(Psi, Amn):
    return Psi@Amn

def get_Hwann(E, Amn):
    return Amn.T.conj()@np.diag(E)@Amn

def scdm_newH(E, psi, nwann, weight):
    psi=psi@np.diag(weight)
    col=scdm(psi.T.conj(), nwann)
    psi_s=psi[col,:]
    Amn=get_Amn(psi_s.T.conj())
    Wann=get_wann(psi, Amn)
    Hwann= get_Hwann(E, Amn)
    return Wann, Hwann

In [120]:
nb=6
H=np.random.random((nb,nb))
H=H+H.T.conj()
H=H+np.diag(np.diag(H))
E, Phi=eigh(H)
Wann, Hwann = scdm_newH(E, Phi, 3)
Ew, Phiw=eigh(Hwann)
print(E)
print(Ew)

TypeError: scdm_newH() missing 1 required positional argument: 'weight'

In [129]:
def Amn_svd(psiT, ncol):
    """
    select columns for a psiT.
    """
    U, S, VT=svd(psiT)
    return U[:, :ncol]


def get_wann(Psi, Amn):
    return Psi@Amn

def get_Hwann(E, Amn):
    return Amn.T.conj()@np.diag(E)@Amn

def scdm_newH_svd(E, psi, nwann, weight):
    psi=psi@np.diag(weight)
    Amn=Amn_svd(psi.T.conj(), nwann)
    Wann=get_wann(psi, Amn)
    Hwann= get_Hwann(E, Amn)
    return Wann, Hwann

nb=6
H=np.random.random((nb,nb))
H=H+H.T.conj()
H=H+np.diag(np.diag(H))
E, Phi=eigh(H)
weight=[0.2,1,0.29,0.3,0,0]
nwann=2
Wann, Hwann = scdm_newH_svd(E, Phi, nwann, weight)
Ew1, Phiw=eigh(Hwann)

Wann, Hwann = scdm_newH(E, Phi, nwann,weight)
Ew2, Phiw=eigh(Hwann)
#print(Wann)

print(E)
print("svd:", Ew1)
print("scdmk", Ew2)

[-0.95920645 -0.86198711  0.77988323  1.24724065  2.20487513  8.20430962]
svd: [-0.86198711  1.24724065]
scdmk [-0.85760709  0.79585973]
